In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import optuna
import math
import warnings


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 1. Загрузка данных

train_x = pd.read_csv("train_x.csv")
train_y = pd.read_csv("train_y.csv")

train_x = train_x.iloc[:, 1:]  # убрать id
train_y = train_y.iloc[:, 1:]

X = train_x.values.astype("float32")
y = train_y["year"].values.astype("float32").reshape(-1, 1)

scaler_X = StandardScaler()
X = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y = scaler_y.fit_transform(y)

dataset = TensorDataset(torch.tensor(X), torch.tensor(y))
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])



In [ ]:
# 2. Модель

class YearPredictor(nn.Module):
    def __init__(self, input_dim, hidden1, hidden2, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden1),
            nn.BatchNorm1d(hidden1),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout),
            
            nn.Linear(hidden1, hidden2),
            nn.BatchNorm1d(hidden2),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout / 2),

            nn.Linear(hidden2, 64),
            nn.LeakyReLU(0.1),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)



In [34]:
# 3. Функция для Optuna
def objective(trial):
    # Гиперпараметры для подбора
    hidden1 = trial.suggest_int("hidden1", 128, 512, step=64)
    hidden2 = trial.suggest_int("hidden2", 64, 256, step=32)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256])
    
    model = YearPredictor(X.shape[1], hidden1, hidden2, dropout).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    best_mse = float("inf")
    patience_counter = 0
    patience = 5
    epochs = 30

    # Cosine LR scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()
        scheduler.step()

        # --- Валидация ---
        model.eval()
        preds_all, targets_all = [], []
        with torch.no_grad():
            for xb, yb in test_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                preds_all.append(preds.cpu())
                targets_all.append(yb.cpu())

        preds_all = torch.cat(preds_all).numpy()
        targets_all = torch.cat(targets_all).numpy()

        preds_orig = scaler_y.inverse_transform(preds_all)
        targets_orig = scaler_y.inverse_transform(targets_all)
        mse = mean_squared_error(targets_orig, preds_orig)

        if mse < best_mse:
            best_mse = mse
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    return best_mse



In [ ]:

# 4. Запуск Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20, timeout=1800, show_progress_bar=True)

print("\n Лучшие параметры:")
for key, value in study.best_params.items():
    print(f"{key}: {value}")
print(f"Лучший MSE: {study.best_value:.3f}")


In [ ]:

# 5. Обучение финальной модели с лучшими параметрами
best_params = study.best_params
model = YearPredictor(X.shape[1],
                      best_params["hidden1"],
                      best_params["hidden2"],
                      best_params["dropout"]).to(device)

optimizer = optim.AdamW(model.parameters(),
                        lr=best_params["lr"],
                        weight_decay=best_params["weight_decay"])
criterion = nn.MSELoss()

train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=best_params["batch_size"], shuffle=False)

epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)

    model.eval()
    preds_all, targets_all = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            preds_all.append(preds.cpu())
            targets_all.append(yb.cpu())

    preds_all = torch.cat(preds_all).numpy()
    targets_all = torch.cat(targets_all).numpy()

    preds_orig = scaler_y.inverse_transform(preds_all)
    targets_orig = scaler_y.inverse_transform(targets_all)
    mse = mean_squared_error(targets_orig, preds_orig)

    print(f"Epoch {epoch+1:02d}/{epochs}, TrainLoss: {total_loss/train_size:.4f}, Test MSE: {mse:.3f}")

print(f"\n Финальный MSE на тесте: {mse:.3f}")


In [ ]:
# 1. Загрузка test_x и подготовка
test_x = pd.read_csv("test_x.csv")

# Отделим id
test_ids = test_x["id"].values
test_x = test_x.iloc[:, :-1] 

X_test = test_x.values.astype("float32")
X_test = scaler_X.transform(X_test)

test_tensor = torch.tensor(X_test).to(device)


# 2. Предсказания модели
model.eval()
with torch.no_grad():
    preds = model(test_tensor).cpu().numpy()

# Обратное преобразование и округление
preds_years = scaler_y.inverse_transform(preds).flatten()
preds_years = np.round(preds_years).astype(int)

# 3. Формирование submission.csv
submission = pd.DataFrame({
    "id": test_ids.astype(int),
    "year": preds_years.astype(int).astype(str) 
})

submission.to_csv("submission.csv", index=False)

print("\n Файл submission.csv успешно сохранён!")
print(submission.head())
